In [ ]:
# Only run once
!pip install cgshop2023-pyutils

In [ ]:
from cgshop2023_pyutils import InstanceDatabase

idb = InstanceDatabase("path/to/zip/or/folder/with/instances")
instance = idb["instance_name"]
print(instance["outer_boundary"])

<h1> Problem visualisation
    